# WRFcast Tutorial

This tutorial will walk through forecast data from your own WRF forecast model data using the wrfcast.py module within pvlib.

This tutorial has been tested against the following package versions:
* Python 
* IPython 
* pandas 
* matplotlib 
* netcdf4 1.4.2 

It should work with other Python and Pandas versions. It requires pvlib >= 0.3.0 and IPython >= 3.0.

Authors:
* Jeffrey Sward (jas983@cornell.edu), Cornell University, November 2019

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# built in python modules
import datetime
import os
import inspect
import sys

# python add-ons
import numpy as np
import pandas as pd
import xarray as xr
import netCDF4
import wrf

# # Import the pvlib module
if sys.platform == 'linux':
    sys.path.append('/home/jsward/Documents/01_Research/01_Renewable_Analysis/WRF/pvlib-python')
import pvlib
from pvlib.wrfcast import WRF

In [ ]:
# Find the absolute file path to your pvlib installation
pvlib_abspath = os.path.dirname(os.path.abspath(inspect.getfile(pvlib)))

# absolute path to WRF data file
datapath = os.path.join(pvlib_abspath, 'data', 'wrfout_d01_2011-01-24_01:00:00')

# Read in the wrfout file using the netCDF4.Dataset method (I think you can also do this with an xarray method)
netcdf_data = netCDF4.Dataset(datapath)
netcdf_data

In [ ]:
# Create an xarray.Dataset from the wrf qurery_variables.
query_variables = [
            'Times',
            'T2',
            'U10',
            'V10',
            'CLDFRA',
            'SWDDNI',
            'SWDDIF'
            ]
first = True
for key in query_variables:
    var = wrf.getvar(netcdf_data, key, timeidx=wrf.ALL_TIMES)
    if first:
        solar_data = var
        first = False
    else:
        solar_data = xr.merge([solar_data, var])

variables = {
            'times': 'times',
            'XLAT': 'lat',
            'XLONG': 'lon',
            'T2': 'temp_air',
            'U10': 'wind_speed_u',
            'V10': 'wind_speed_v',
            'CLDFRA': 'total_clouds',
            'SWDDNI': 'dni',
            'SWDDIF': 'dhi' 
            }
solar_data = xr.Dataset.rename(solar_data, variables)
times = solar_data.times
ntimes = solar_data.sizes['Time']
nlat = solar_data.sizes['south_north']
nlon = solar_data.sizes['west_east']
solar_data

In [ ]:
# Explore how the WRF forecast model behaves
fm = WRF()
wind_speed = fm.uv_to_speed(solar_data)
temp_air = fm.kelvin_to_celsius(solar_data['temp_air'])
# ghi = fm.dni_and_dhi_to_ghi(solar_data['dni'], solar_data['dhi'])

In [ ]:
# Convert xarray Datasets to a pandas DataFrames
solar_data = solar_data.to_dataframe()
times = times.to_dataframe()
solar_data

In [ ]:
# Run the solar position algorithm for time, lat, and lon indices, and concatonate them into a single DataFrame
numerical_time_indices = range(0, ntimes)
lat_indices = range(0, nlat)
lon_indices = range(0, nlon)
first = True
for num_time_idx in numerical_time_indices:
    time = times.index.get_level_values('Time')[num_time_idx]
    print(f'Time Index: {time}')
    for lat_idx in lat_indices:
        for lon_idx in lon_indices:
            
#             print(f'Time Index: {time}')
#             print(f'\tLatitude index: {lat_idx}')
#             print(f'\t\tLongitude index: {lon_idx}')
            
            solpos_new = pvlib.solarposition.spa_xarray_python(time, 
                                        solar_data['lat'].loc[time, 0, lat_idx, lon_idx], 
                                        solar_data['lon'].loc[time, 0, lat_idx, lon_idx], lat_idx, lon_idx)
            
            if first:
                solpos = solpos_new
                first = False
            else:
                solpos = pd.concat([solpos, solpos_new])
            

In [ ]:
solpos

In [ ]:
solpos = solpos.to_dataframe()
solar_data['zenith'] = solpos['zenith']
solar_data = solar_data.to_xarray()

In [ ]:
solpos

In [ ]:
new_filename_1 = './solpos_d01_2011-01-24_01:00:00.nc'
solpos.to_netcdf(path=new_filename_1)